In [2]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings('ignore')
import boto3
import awswrangler

In [ ]:
sparker = SparkSession.builder.appName("stream reading"). \
                getOrCreate()

In [ ]:
### Creating sparksession that reads from the stream

read_lines = sparker.readStream.format("socket"). \
            option("host","localhost"). \
            option("port",9999). \
            load()

In [ ]:
read_lines.printSchema()

In [ ]:
counting_sheep = read_lines.select(split("value",' ').alias('words')). \
                    groupBy('words'). \
                    agg(count('words').alias('word_count'))

In [ ]:
counting_sheep.printSchema()

In [ ]:
writing_count = counting_sheep.writeStream \
            .format('console') \
            .outputMode('update') \
            .start()

writing_count.awaitTermination()

In [ ]:
practiceList = [
    [1,"This line is the first of many"],
    [2,"Machine is talking a lot"],
    [3,"Making Machines speak has been the passion for many"]
]
schema_df = ["id int","line string"]
practiceDF = sparker.createDataFrame(data=practiceList,schema=schema_df)

In [ ]:
practiceDF.printSchema()

In [ ]:
count_sample.show()

In [ ]:
sparker.stop()

In [ ]:
newSparker = SparkSession \
    .builder \
    .appName("Streaming Process Files") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .master("local[*]") \
    .getOrCreate()

In [ ]:
newSparker.conf.set("spark.sql.streaming.schemaInference", True)
newSparker.conf.set("spark.sql.adaptive.enabled",False)

In [ ]:
### Creating sparksession that reads from the stream

read_lines = newSparker.readStream.format("socket"). \
            option("host","localhost"). \
            option("port",9999). \
            load()

In [ ]:
#Now we can count the sheep jumping in the spark stream

count_sheep_1 = read_lines.select(explode(split('value',' ')).alias('words')). \
                        groupBy('words'). \
                        agg(count('words').alias('count_words'))
count_sheep_1.printSchema()

In [ ]:
#To write the output to file watermarking is required

count_sheep_1 = read_lines.select(explode(split('value',' ')).alias('words')). \
                        groupBy('words'). \
                        agg(count('words').alias('count_words'))
count_sheep_1.printSchema()

In [ ]:
writeDf = count_sheep_1.writeStream.format("memory"). \
        queryName('stream_table'). \
        outputMode("update"). \
        start()

In [ ]:
# Next lets get the output from the table

newSparker.sql("SELECT * FROM stream_table").show()

In [ ]:
from datetime import datetime

In [ ]:
newtime = datetime.now().strftime('%H:%M')

In [ ]:
newtime

In [ ]:
# Lets see if for each batch execution can be done
count_sheep_1 = read_lines.select(explode(split('value',' ')).alias('words')). \
                        groupBy('words'). \
                        agg(count('words').alias('count_words')). \
                        withColumn('time_date', lit(datetime.now().strftime('%H:%M')))
count_sheep_1.printSchema()

In [ ]:
writeDf = count_sheep_1.writeStream.format("memory"). \
        queryName('time_table'). \
        outputMode("update"). \
        start()

In [ ]:
writeDf = count_sheep_1.writeStream.format("memory"). \
        queryName('wat_mar'). \
        outputMode("update"). \
        start()

In [ ]:
newSparker.sql("SELECT * FROM time_table").show()

In [ ]:
import seaborn as sns

The chart updates after the new data is taken from the table and fed into the pandas dataframe. It is not real time streaming

In [ ]:
chartingData = newSparker.sql("SELECT * FROM time_table").to_pandas_on_spark()

In [ ]:
chartingData = chartingData[chartingData.words != '']

In [ ]:
import matplotlib as plt

plt.pyplot.bar(x=chartingData['words'].to_numpy(),
               height=chartingData['count_words'].to_numpy())

In [ ]:
#Before trying the file data ingestion, want to check water marking

# Lets see if for each batch execution can be done
count_sheep_wm = read_lines.select(explode(split('value',' ')).alias('words')). \
                        groupBy('words'). \
                        agg(count('words').alias('count_words')). \
                        withColumn('time_date', lit(datetime.now().strftime('%H:%M'))). \
                        withColumn('time_stamp', lit(datetime.now())). \
                        withWatermark('time_stamp','1 seconds')
count_sheep_wm.printSchema()

In [ ]:
count_sheep_wm.writeStream.format("csv"). \
            outputMode("append"). \
            option("checkpointLocation","checkpoint_dir"). \
            option("path","samplespace/"). \
            start()

In [ ]:
def foreach_batch_function(df, epoch_id):
    df.where("count_words > 1")

In [ ]:
count_sheep_wm.writeStream. \
            foreach(foreach_batch_function). \
            outputMode('')
            start()

In [ ]:
fileData= newSparker.readStream.format('csv'). \
    option("cleanSource","archive"). \
    option("sourceArchiveDir","checkpoint_dir"). \
    option("maxFilesPerTrigger",1). \
    load('samplesource',)

In [ ]:
selectedData = fileData.selectExpr("_c0 as date","_c2 as borough",
                                   "_c3 as zipcode")

In [ ]:
selectedData.writeStream.outputMode("append").format('console') \
        .start()

In [ ]:
%%sh
export PYSPARK_SUBMIT_ARGS="--master spark://192.168.2.40:7077"
#export PYSPARK_SUBMIT_ARGS="--master local[*] pyspark-shell"

In [ ]:
%%sh
echo $PYSPARK_SUBMIT_ARGS

In [3]:
spark = SparkSession \
    .builder \
    .appName("Streaming from Kafka") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0') \
    .config("spark.sql.shuffle.partitions", 4) \
    .master("local[*]") \
    .getOrCreate()

23/01/26 17:02:27 WARN Utils: Your hostname, codeStation resolves to a loopback address: 127.0.1.1; using 192.168.78.83 instead (on interface wlo1)
23/01/26 17:02:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/solverbot/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/solverbot/.ivy2/cache
The jars for the packages stored in: /home/solverbot/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-04b6dea8-1441-42c8-80c8-2e53db3f5377;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
d

23/01/26 17:03:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [29]:
streaming_df = spark.readStream\
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "local_test") \
    .option("startingOffsets", "earliest") \
    .load()

In [21]:
streaming_df = spark.read\
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "local_test") \
    .option("startingOffsets", "earliest") \
    .load()

In [30]:
streaming_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [31]:
from pyspark.sql.types import StringType, StructField, StructType, DecimalType

json_schema = StructType([StructField('Date', StringType(), True), \
StructField('Open', DecimalType(), True), \
StructField('High', DecimalType(), True), \
StructField('Low', DecimalType(), True), \
StructField('Close', DecimalType(), True), \
StructField('Adj Close', DecimalType(), True), \
StructField('Adj Close', DecimalType(), True)])                          

In [32]:
# Parse value from binay to string
json_df = streaming_df.selectExpr("cast(value as string) as value")

# Apply Schema to JSON value column and expand the value
from pyspark.sql.functions import from_json

json_expanded_df = json_df.withColumn("value", from_json(json_df["value"], json_schema)).select("value.*") 

In [25]:
json_expanded_df.show(10)

+----------+----+----+---+-----+---------+---------+
|      Date|Open|High|Low|Close|Adj Close|Adj Close|
+----------+----+----+---+-----+---------+---------+
|2017-06-21| 156| 156|153|  154|     null|      148|
|2015-02-27| 161| 162|160|  162|     null|      141|
|2013-08-07| 190| 190|188|  189|     null|      158|
|2017-12-21| 153| 153|151|  152|     null|      148|
|2014-03-10| 188| 188|186|  186|     null|      158|
|2013-05-31| 209| 212|208|  208|     null|      174|
|2016-07-14| 159| 161|159|  160|     null|      148|
|2015-02-10| 157| 159|155|  159|     null|      138|
|2014-09-17| 193| 194|192|  193|     null|      166|
|2016-06-16| 150| 151|149|  151|     null|      140|
+----------+----+----+---+-----+---------+---------+
only showing top 10 rows



In [33]:
filter_df = json_expanded_df.where("Open > 150") \
    .withColumn("tradeDate", to_date("Date", "yyyy-MM-dd")) \
    .select("Open","High","Low")

In [28]:
# Check the schema of the agg_df, post a kafka message and change readStream to read 
filter_df.printSchema()
filter_df.show()

root
 |-- Open: decimal(10,0) (nullable = true)
 |-- High: decimal(10,0) (nullable = true)
 |-- Low: decimal(10,0) (nullable = true)

+----+----+---+
|Open|High|Low|
+----+----+---+
| 156| 156|153|
| 161| 162|160|
| 190| 190|188|
| 153| 153|151|
| 188| 188|186|
| 209| 212|208|
| 159| 161|159|
| 157| 159|155|
| 193| 194|192|
| 156| 156|154|
| 161| 161|160|
| 198| 198|193|
| 178| 178|177|
| 177| 178|176|
| 191| 194|191|
| 180| 181|179|
| 183| 185|183|
| 175| 178|175|
| 155| 155|153|
| 194| 195|191|
+----+----+---+
only showing top 20 rows



In [36]:
writing_df = filter_df.writeStream \
    .format("console") \
    .option("checkpointLocation","checkpoint_dir") \
    .outputMode("append") \
    .start()

23/01/26 17:55:36 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 156| 156|153|
| 161| 162|160|
| 190| 190|188|
| 153| 153|151|
| 188| 188|186|
| 209| 212|208|
| 159| 161|159|
| 157| 159|155|
| 193| 194|192|
| 156| 156|154|
| 161| 161|160|
| 198| 198|193|
| 178| 178|177|
| 177| 178|176|
| 191| 194|191|
| 180| 181|179|
| 183| 185|183|
| 175| 178|175|
| 155| 155|153|
| 194| 195|191|
+----+----+---+
only showing top 20 rows



-------------------------------------------
Batch: 1
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 165| 166|164|
| 182| 183|181|
| 191| 192|191|
| 182| 184|182|
| 168| 170|168|
| 157| 157|154|
| 167| 169|167|
+----+----+---+

-------------------------------------------
Batch: 2
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 174| 175|174|
| 160| 162|160|
| 167| 168|166|
+----+----+---+

-------------------------------------------
Batch: 3
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 153| 153|152|
+----+----+---+

-------------------------------------------
Batch: 4
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 156| 157|153|
+----+----+---+

-------------------------------------------
Batch: 5
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 182| 183|182|

-------------------------------------------
Batch: 6
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 152| 153|150|
+----+----+---+

-------------------------------------------
Batch: 7
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 154| 154|153|
+----+----+---+

-------------------------------------------
Batch: 8
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 188| 188|186|
+----+----+---+

-------------------------------------------
Batch: 9
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
+----+----+---+

-------------------------------------------
Batch: 10
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 152| 153|152|
+----+----+---+

-------------------------------------------
Batch: 11
-------------------------------------------
+----+----+---+
|Open|High|

-------------------------------------------
Batch: 18
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 151| 153|151|
+----+----+---+

-------------------------------------------
Batch: 19
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 174| 175|174|
+----+----+---+

-------------------------------------------
Batch: 20
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 172| 175|172|
+----+----+---+



-------------------------------------------
Batch: 21
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 151| 151|149|
+----+----+---+

-------------------------------------------
Batch: 22
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 158| 158|155|
+----+----+---+



-------------------------------------------
Batch: 23
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 151| 153|151|
+----+----+---+

-------------------------------------------
Batch: 24
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 154| 156|154|
+----+----+---+

-------------------------------------------
Batch: 25
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 192| 192|190|
+----+----+---+

-------------------------------------------
Batch: 26
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 158| 161|158|
+----+----+---+

-------------------------------------------
Batch: 27
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
+----+----+---+

-------------------------------------------
Batch: 28
-------------------------------------------
+----+----+---+
|Open|H

-------------------------------------------
Batch: 31
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 162| 163|160|
+----+----+---+



-------------------------------------------
Batch: 32
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 161| 162|160|
+----+----+---+



-------------------------------------------
Batch: 33
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 189| 190|189|
+----+----+---+

-------------------------------------------
Batch: 34
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 164| 164|163|
+----+----+---+

-------------------------------------------
Batch: 35
-------------------------------------------
+----+----+---+
|Open|High|Low|
+----+----+---+
| 190| 190|189|
+----+----+---+

